In [7]:
import  cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [8]:
import pygame

pygame.mixer.init()

capture_sound = pygame.mixer.Sound("capture.wav")
open_sound = pygame.mixer.Sound("open.wav")

In [9]:
(train_images, test_images) = (train_images / 255, test_images / 255)
model = keras.Sequential([
    keras.layers.Input(shape=(28, 28)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(56, activation='relu'),
    keras.layers.Dense(10, activation='softmax') #분류 종류가 3개 이상일 때 사용
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
model.save("my_first_DNN_model.keras")
load_model=keras.models.load_model("my_first_DNN_model.keras")

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9368 - loss: 0.2134
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9722 - loss: 0.0930
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9793 - loss: 0.0673
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9825 - loss: 0.0521
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9866 - loss: 0.0404
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9890 - loss: 0.0321
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9904 - loss: 0.0302
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9918 - loss: 0.0250
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9932 - loss: 0.0222
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9939 - loss: 0.0192
313/313 - 0s - 1ms/step - accuracy: 0.9788 - loss: 0.0991


In [10]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

digits = ['-', '-', '-', '-']  # 4자리 숫자 상태
digit_index = 0                # 다음에 채울 위치
TARGET_CODE = ['2', '7', '3', '1']  # 원하는 4자리 숫자
success = False
door_alpha = 0.0
door_opening = False

door=cv2.imread("closed_door.png")
# 문 이미지 불러오기
img1 = cv2.imread('closed_door.png')  # 닫힌 문 이미지
img2 = cv2.imread('opened_door.png')    # 열린 문 이미지

while True:
    if door_opening:
        door_alpha += 0.03   # 속도 조절 (작을수록 천천히)
        if door_alpha >= 1.0:
            door_alpha = 1.0
            door_opening = False  # 애니메이션 종료

    door = cv2.addWeighted(img1, 1 - door_alpha,img2, door_alpha,0)

    cv2.imshow("Door", door)
    ret, frame = cap.read()
    if not ret:
        print("프레임을 가져올 수 없습니다.")
        break
    flip_frame=cv2.flip(frame,1)
    height, width, _ = frame.shape
    center_x, center_y = width//2, height//2
    roi = flip_frame[center_y - 150:center_y + 150, center_x - 150:center_x + 150]
    cv2.rectangle(flip_frame, (center_x - 150, center_y - 150), (center_x + 150, center_y + 150), (0, 0, 255), 2)

    display_text = ' '.join(digits)
    cv2.putText(flip_frame,display_text,(10, height - 10), cv2.FONT_HERSHEY_SIMPLEX,1.2,(0, 255, 0),2,cv2.LINE_AA) #왼쪽 아래에 입력된 숫자 표시
    if success:
        for alpha in np.linspace(0, 1, 30):  # 0에서 1까지 30단계로 나눔
    # 가중치 계산: (1-alpha)*첫번째 이미지 + alpha*두번째 이미지
            door = cv2.addWeighted(img1, 1 - alpha, img2, alpha, 0)


    cv2.imshow("Webcam", flip_frame)
    #화면 캡쳐를 위한 키 값 받기
    key=cv2.waitKey(1) & 0xFF
    if key == ord('c') or key == ord('C'):
        gray_image=cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray_image=np.flip(gray_image, 1)
        cv2.imwrite("gray_image.png", gray_image)
        gaussian_blur = cv2.GaussianBlur(gray_image, (5, 5), 3)
        #이진화
        _, otsu_thread = cv2.threshold(gaussian_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        ###########Morph
        kernel=np.ones((5,5), np.uint8)
        erosion=cv2.erode(otsu_thread, kernel, iterations=3) #글씨 두껍게 해줌
        cv2.imwrite("digit_binary_image.png", erosion)
        #이미지 자르기
        img=cv2.imread("digit_binary_image.png", cv2.IMREAD_UNCHANGED)
        h, w = img.shape[:2]
        crop_size = 280
        cx, cy = w//2, h//2
        half = crop_size//2
        x1, x2 = cx-half, cx+half
        y1, y2 = cy-half, cy+half
        #경계면 설정
        x1 = max(0,x1)
        y1 = max(0,y1)
        x2 = min(w,x2)
        y2 = min(h,y2)

        cropped_img = img[y1:y2, x1:x2]

        #이미지 반전
        reversed_img = cv2.bitwise_not(cropped_img)
        cv2.imwrite("IMAGE_FOR_TEST.png", reversed_img)

        #이미지 축소 (28x28)
        resized_img = cv2.resize(reversed_img, (28, 28))
        #정규화
        resized_img = resized_img / 255.0

        predict_image=model.predict(resized_img[np.newaxis, :, :]) #차원 바꿔줌

        #예측 숫자 출력
        print(predict_image.argmax())
        predicted_digit = np.argmax(predict_image)
        capture_sound.play()

        if digit_index < 4:
            digits[digit_index] = str(predicted_digit)
            digit_index += 1

        if digit_index == 4:
            if digits == TARGET_CODE and not door_opening:
                success = True
                door_opening = True
                open_sound.play()
            else:
                success = False
                digits = ['-', '-', '-', '-']
                digit_index = 0

    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1
